# Data_Logger_1[Keyboard]

In [3]:
# Date : 07.04.2020
# version : 1.0.200407
# > save data to a csv file with key stroke ('w')
# > display amplitude readings from microphones and last recorded data on the scv file
# > Stop the program and change the 'direction' before moving to the next region

import csv
import pyaudio
import numpy as np
import keyboard
import time
from IPython.display import clear_output

direction = 5  # L=0, R=1, F=2, B=3, U=4, D=5

CHUNK = 1024
RATE = 44100
CHANNELS = 2
FORMAT = pyaudio.paInt16

p=pyaudio.PyAudio()
stream=p.open(format=FORMAT,
              channels=CHANNELS,
              rate=RATE,
              input=True, 
              frames_per_buffer=CHUNK
             )

def MAD(data): # Maximum Amplitude Displacement in a CHUNK lengthed window
    amp = np.abs(np.max(data)-np.min(data))
    return amp

while True:
    data = np.fromstring(stream.read(CHUNK),dtype=np.int16)   
    data_L = data[0::2]  # select even indices(left channel)
    data_R = data[1::2]  # select odd indices(right channel)

    left = MAD(data_L)
    right = MAD(data_R)

    print('L:{:06} R:{:06}'.format(left,right))

    with open('data.csv', "r") as infile:     # Print the last recorded value 
        read = csv.reader(infile)
        data = list(read)
        idx=len(data)
        print('Last record:','[ ID =',idx,']',data[idx-1])

    clear_output(wait=True)

    if keyboard.is_pressed('Esc'):
        stream.stop_stream()
        stream.close()
        p.terminate()
        break

    if keyboard.is_pressed('w'):              # write to CSV
        row = [left, right, direction]
        with open('data.csv', 'a', newline='') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
        time.sleep(.5)                        # add a delay for skipping continuous values

L:007954 R:007233
Last record: [ ID = 150 ] ['26437', '26621', '5']


# Data_Logger_2 [Pymata4]

- Need Python 3.7 for firmata4 - use the kernel | Python[conda env:tensorflow_2]
- pymata4 v1.1
- To install 'Firmata Express' on arduino follow [this](https://mryslab.github.io/pymata4/firmata_express).(*Need to install 'Ultrasonic' library*)

In [ ]:
# Date : 09.04.2020
# version : 0.1.200409
# > save data to a csv file with key stroke ('w') or the button the press on the "sound source"
# > display amplitude readings from microphones and last recorded data on the scv file
# > Stop the program and change the 'direction' before moving to the next region

import csv
import pyaudio
import numpy as np
import time
import keyboard
from pymata4 import pymata4
from IPython.display import clear_output

direction = 5  # L=0, R=1, F=2, B=3, U=4, D=5

DIGITAL_PIN = 2
TONE_PIN = 3

CHUNK = 1024
RATE = 44100
CHANNELS = 2
FORMAT = pyaudio.paInt16

p = pyaudio.PyAudio()
stream = p.open(format = FORMAT,
                channels = CHANNELS,
                rate = RATE,
                input = True, 
                frames_per_buffer = CHUNK)

def MAD(data): # Maximum Amplitude Displacement in a CHUNK lengthed window
    amp = np.abs(np.max(data)-np.min(data))
    return amp

def the_callback(data):
    if data[2]:                                # If the value of the button press is HIGH
        board.play_tone(TONE_PIN, 256, 500)
        time.sleep(.1)                         # debounce/settle time
        row = [left, right, direction]
        with open('data.csv', 'a', newline='') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
        
        
board = pymata4.Pymata4()
board.set_pin_mode_tone(TONE_PIN)
board.set_pin_mode_digital_input(DIGITAL_PIN, callback = the_callback)

while True:
    data = np.fromstring(stream.read(CHUNK),dtype=np.int16)   
    data_L = data[0::2]  # select even indices(left channel)
    data_R = data[1::2]  # select odd indices(right channel)

    left = MAD(data_L)
    right = MAD(data_R)

    print('L:{:06} R:{:06}'.format(left,right))

    with open('data.csv', "r") as infile:     # Print the last recorded value 
        read = csv.reader(infile)
        data = list(read)
        idx=len(data)
        print('Last record:','[ ID =',idx,']',data[idx-1])

    clear_output(wait=True)
    
    if keyboard.is_pressed('w'):              # write to CSV
        row = [left, right, direction]
        with open('data.csv', 'a', newline = '') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
        time.sleep(.5)  
    
    if keyboard.is_pressed('Esc'):
        stream.stop_stream()
        stream.close()
        p.terminate()
        board.shutdown()
        break

# Data_Logger_3[pymata4][averaging]

In [ ]:
# Date : 11.04.2020
# version : 0.2.200411
# Simple averaging method
# > bug: when the position of the sound source change, 
#        the first 5-6 values sometimes remain cloase to the previous levels

import csv
import pyaudio
import numpy as np
import time
import keyboard
from pymata4 import pymata4
from IPython.display import clear_output

direction = 1  # L=0, R=1, F=2, B=3, U=4, D=5

DIGITAL_PIN = 2
TONE_PIN = 3

count = 22     # Since the RATE=44100 and CHUNK=1024, the maximum allowed count = 43/second

CHUNK = 1024
RATE = 44100
CHANNELS = 2
FORMAT = pyaudio.paInt16

p = pyaudio.PyAudio()
stream = p.open(format = FORMAT,
                channels = CHANNELS,
                rate = RATE,
                input = True, 
                frames_per_buffer = CHUNK)

def the_callback(data):
    if data[2]:                                # If the value of the button press is HIGH
        totL = totR = 0
        board.play_tone(TONE_PIN, 256, 1000)
        clear_output(wait = True)
        for i in range(count):
            data = np.fromstring(stream.read(CHUNK),dtype = np.int16)
            rawL = data[0::2]
            rawR = data[1::2]            
            print(np.abs(np.max(rawL)),'-', np.abs(np.max(rawR)))
            totL += np.abs(np.max(rawL))
            totR += np.abs(np.max(rawR)) 
        left = totL/count
        right = totR/count    
        
        row = [left, right, direction]
        with open('data.csv', 'a', newline = '') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
            
        with open('data.csv', "r") as infile:     # Print the last recorded value 
            read = csv.reader(infile)
            data = list(read)
            idx=len(data)
            print('Last record:','[ ID =',idx,']',data[idx-1])
             
board = pymata4.Pymata4()
board.set_pin_mode_tone(TONE_PIN)
board.set_pin_mode_digital_input(DIGITAL_PIN, callback = the_callback)

while True:      
    if keyboard.is_pressed('Esc'):
        stream.stop_stream()
        stream.close()
        p.terminate()
        board.shutdown()
        break